In [86]:
#import libraries:
import time
import requests
import pandas as pd
import googlemaps 
import win32com.client
import random

#Helpful threads for using Google Maps API:
# https://stackoverflow.com/questions/13070477/how-can-i-get-more-than-60-results-from-the-google-places-api   <-- info about retrieving more than 60 places
# https://hackernoon.com/how-i-sort-of-got-around-the-google-maps-api-results-limit-1c673e66ef36

In [118]:
#API Key

API_Key = "insert API Key Here"

#Run googlemaps key
map_client = googlemaps.Client(API_Key)


#Option 1:
    #-Already know where to search for restaurants:
    
#Places we want to look for restaurants:

places = ["Lisbon", "Sintra", "Cascais", "Porto"]

#functions to get places:

def get_place_info(location_name):
    try:
        response = map_client.places(query=location_name)
        results = response.get("results")
        return results 

    except:
        print("could not find")
        return None


In [119]:
#Option 2:
    #-Ask the user to input the place where they want to search for restaurants:

result_list = get_place_info(input("search for a place in Portugal: "))

search for a place in Portugal:  Ironhack


In [121]:
#For the locations given, this function saves the latitude and longitude of the place.

def get_latlog(result_list):
    locations_data = []
    lat_log = []
    final_latlog = []
    for information in result_list[0].values():
        locations_data.append(information)
    
    for info_loc in locations_data[2].values():
        lat_log.append(info_loc)
        
    for lat_log_values in lat_log[0].values():
        final_latlog.append(lat_log_values)
    return final_latlog

lati_logi_info = get_latlog(result_list)


In [122]:
#Search near_by restaurants:
    #Here we can input what we want to search for, the radius distance given a latitude and longitude 
map_client = googlemaps.Client(API_Key)
lat_log = (lati_logi_info[0],lati_logi_info[1])
search_string = "restaurante"
distance = 750
business_list = []

response = map_client.places_nearby(
location = lat_log,
type = "restaurant",
radius = distance,
)

business_list.extend(response.get("results"))
next_page_token = response.get("next_page_token")

while next_page_token:
    time.sleep(random.random()+2)
    response = map_client.places_nearby(
        location = lat_log,
        type = "restaurant",
        radius = distance,
        page_token = next_page_token
    )
    business_list.extend(response.get("results"))
    next_page_token = response.get("next_page_token")

In [123]:
#Create a dataframe from the results:

maps_df = pd.DataFrame(business_list)
maps_df["url"] = "https://google.com/maps/place/?q=place_id:" + maps_df["place_id"]

In [124]:
#Check how many restaurants were capturated and show 5 first rows of the dataframe:
print(maps_df.shape)
maps_df.head(5)

(60, 19)

In [13]:
#Now All together in the same kernel:

#import libraries:
import time
import pandas as pd
import googlemaps 
import random

#My API_Key:
API_Key = "insert API Key Here"

#Running googlemaps key:
map_client = googlemaps.Client(API_Key)

#Function to get places:
def get_place_info(location_name):
    try:
        response = map_client.places(query=location_name)
        results = response.get("results")
        return results 

    except:
        print("could not find")
        return None

#Ask the user to input a place address or city, searching the input on the get_place_info function:
result_list = get_place_info(input("search for a place/address/city in Portugal: "))


#Function to get the longitude and latitude of the place we chose in result_list:    
def get_latlog(result_list):
    locations_data = []
    lat_log = []
    final_latlog = []
    for information in result_list[0].values():
        locations_data.append(information)
    
    for info_loc in locations_data[2].values():
        lat_log.append(info_loc)
        
    for lat_log_values in lat_log[0].values():
        final_latlog.append(lat_log_values)
    return final_latlog

lati_logi_info = get_latlog(result_list)


"""Next we will Search for restaurants that are in a X meter distance 
from the place we chose, by getting the latitude and longitude from
get_latlog function: """

def restaurants_nearby(lati_logi_info):
    map_client = googlemaps.Client(API_Key)
    lat_log = (lati_logi_info[0],lati_logi_info[1])
#    search_string = input("what kind of food would you like to eat there?")
    distance = input("What is your radius distance in meters?")
    while int(distance) > 2000:
            display("Ohh, don't walk that much, choose a distance inferior of 2000 m")
            distance = input("What is your radius distance in meters?")   
    business_list = []

    response = map_client.places_nearby(
    location = lat_log,
#    keyword = search_string,
    type = "restaurant",
    radius = distance,
    )

    business_list.extend(response.get("results"))
    next_page_token = response.get("next_page_token")

    while next_page_token:
        time.sleep(random.random()+2)
        response = map_client.places_nearby(
        location = lat_log,
#        keyword = search_string,
        type = "restaurant",
        radius = distance,
        page_token = next_page_token
    )
        business_list.extend(response.get("results"))
        next_page_token = response.get("next_page_token")
    
    return business_list

#Display maps_df table:
maps_df = pd.DataFrame(restaurants_nearby(lati_logi_info))
maps_df["url"] = "https://google.com/maps/place/?q=place_id:" + maps_df["place_id"]
display(maps_df.head(10))

search for a place/address/city in Portugal:  terreiro do paço
What is your radius distance in meters? 1000


,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,name,opening_hours,photos,place_id,plus_code,rating,reference,scope,types,user_ratings_total,vicinity,price_level,permanently_closed,url
0,OPERATIONAL,"{'location': {'lat': 38.7130556, 'lng': -9.139...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,The Art Inn Lisbon,{'open_now': True},"[{'height': 2268, 'html_attributions': ['<a hr...",ChIJITS6qng0GQ0RYMBD2MNlQwE,"{'compound_code': 'PV76+62 Lisbon, Portugal', ...",4.3,ChIJITS6qng0GQ0RYMBD2MNlQwE,GOOGLE,"[cafe, lodging, bar, restaurant, food, point_o...",176,"Rua 1º de Dezembro 31, Lisboa",NaN,NaN,https://google.com/maps/place/?q=place_id:ChIJ...
1,OPERATIONAL,"{'location': {'lat': 38.7115028, 'lng': -9.138...",https://maps.gstatic.com/mapfiles/place_api/ic...,#909CE1,https://maps.gstatic.com/mapfiles/place_api/ic...,Brown's Central Hotel,{'open_now': True},"[{'height': 3264, 'html_attributions': ['<a hr...",ChIJiULG6Hg0GQ0REhiOp3X8fYA,"{'compound_code': 'PV66+JM Lisbon, Portugal', ...",4.5,ChIJiULG6Hg0GQ0REhiOp3X8fYA,GOOGLE,"[lodging, bar, restaurant, food, point_of_inte...",827,"Rua dos Sapateiros 144, Lisboa",NaN,NaN,https://google.com/maps/place/?q=place_id:ChIJ...
2,OPERATIONAL,"{'location': {'lat': 38.70972219999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Clube de Fado - Fado & Food Group,{'open_now': False},"[{'height': 5247, 'html_attributions': ['<a hr...",ChIJV-jS2nY0GQ0Rl2LL675xCYE,"{'compound_code': 'PV59+VH Lisbon, Portugal', ...",4.2,ChIJV-jS2nY0GQ0Rl2LL675xCYE,GOOGLE,"[restaurant, food, point_of_interest, establis...",1174,"Rua de São João da Praça 86, Lisboa",3.0,NaN,https://google.com/maps/place/?q=place_id:ChIJ...
3,OPERATIONAL,"{'location': {'lat': 38.71009939999999, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Belcanto,{'open_now': False},"[{'height': 608, 'html_attributions': ['<a hre...",ChIJITj75340GQ0RXULVgWMFloA,"{'compound_code': 'PV65+2C Lisbon, Portugal', ...",4.6,ChIJITj75340GQ0RXULVgWMFloA,GOOGLE,"[restaurant, food, point_of_interest, establis...",857,"Rua Serpa Pinto 10A, Lisboa",4.0,NaN,https://google.com/maps/place/?q=place_id:ChIJ...
4,CLOSED_TEMPORARILY,"{'location': {'lat': 38.7124655, 'lng': -9.142...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Cervejaria Trindade,NaN,"[{'height': 1367, 'html_attributions': ['<a hr...",ChIJbymNeX80GQ0ROXnZMyq-WxM,"{'compound_code': 'PV65+X2 Lisbon, Portugal', ...",4.1,ChIJbymNeX80GQ0ROXnZMyq-WxM,GOOGLE,"[restaurant, bar, food, point_of_interest, est...",4726,"Rua Nova da Trindade 20 C, Lisboa",2.0,True,https://google.com/maps/place/?q=place_id:ChIJ...
5,OPERATIONAL,"{'location': {'lat': 38.709134, 'lng': -9.1333...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Taberna Moderna,{'open_now': False},"[{'height': 998, 'html_attributions': ['<a hre...",ChIJ8USpSXc0GQ0RAIMnuSbwXp8,"{'compound_code': 'PV58+MM Lisbon, Portugal', ...",4.4,ChIJ8USpSXc0GQ0RAIMnuSbwXp8,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",825,"Rua dos Bacalhoeiros 18A, Lisboa",2.0,NaN,https://google.com/maps/place/?q=place_id:ChIJ...
6,OPERATIONAL,"{'location': {'lat': 38.7086706, 'lng': -9.135...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,Martinho da Arcada,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJIcori3k0GQ0RquwgxxH1IN0,"{'compound_code': 'PV57+FM Lisbon, Portugal', ...",4.2,ChIJIcori3k0GQ0RquwgxxH1IN0,GOOGLE,"[tourist_attraction, restaurant, food, point_o...",1174,"Praça do Comércio 3, Lisboa",2.0,NaN,https://google.com/maps/place/?q=place_id:ChIJ...
7,OPERATIONAL,"{'location': {'lat': 38.710891, 'lng': -9.1294...",https:/

In [14]:
#Saving the dataframe into a CSV:

maps_df.to_csv(r"C:\Users\crocs\OneDrive - Universidade de Lisboa\Ambiente de Trabalho\Gonçalo\College&Courses\Data Analytics- IronHack\Course Data Analytics\Projects\Project3\Project3\google_maps.csv")